<a href="https://colab.research.google.com/github/arquansa/PSTB-exercises/blob/main/Week09/Day5/DC5/W9D5DC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Daily Challenge: Building an Agent with LangGraph and the Gemini API#

- How to create a stateful application using LangGraph.
- How to integrate Gemini API (via LangChain) into your LangGraph application.
- How to define and manipulate state using TypedDict and node functions.
- How to simulate dynamic, tool-augmented behavior (like menus and ordering) in a conversational loop.
- How to model conditional transitions, loops, and user interaction using LangGraph.

A conversational cafe ordering system called BaristaBot. This bot will:
- Use natural language to take coffee/tea orders.
- Offer a real-time menu using a tool.
- Confirm and modify orders.
- Loop through conversation until an order is placed.
- Handle tool calls using LangGraph’s ToolNode mechanism.


✅ Challenge Goal - Create a LangGraph-based BaristaBot that:

- Takes natural language coffee/tea orders
- Displays a real-time menu using a tool
- Confirms or modifies orders
- Loops until the order is confirmed
- Uses Gemini API as the LLM via LangChain

Sample Interaction

- You: Can I see the menu?
- BaristaBot: ☕ Coffee Menu: ...
- You: I'll take a cappuccino.
- BaristaBot: You ordered a Cappuccino. Type 'confirm' to place it or 'change' to modify.
- You: confirm

✅ Order placed.

#1. Setup Requirements
Install necessary packages:

In [ ]:
pip install langgraph langchain-google-genai

In [ ]:
import os
os.environ['GOOGLE_API_KEY'] = "your-gemini-api-key"

#2. Define conversation state with TypedDict

Create a shared state to track conversation context, order info, and control flags:

In [ ]:
from typing import TypedDict, List, Optional

class BaristaState(TypedDict):
    messages: List[str]
    order: Optional[str]
    confirmed: bool
    menu_shown: bool
    input_text: Optional[str]

#3. Create Menu Tool#


Use a tool to return a mock menu dynamically:

In [ ]:
def get_menu() -> str:
    return """☕ Coffee Menu:
- Espresso
- Cappuccino
- Latte

🍵 Tea Menu:
- Green Tea
- Chai
- Earl Grey"""

#Wrap it using LangChain's tool mechanism:

from langchain.tools import Tool

menu_tool = Tool(
    name="get_menu",
    func=lambda _: get_menu(),
    description="Returns the coffee and tea menu."
)

#4. Setup Gemini LLM"

Integration of Gemini, via LangChain into the LangGraph application.  

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)

#5. Define Node Functions#

These represent your LangGraph steps: the different tasks or actions accomplished by the BaristaBot.

Parse User Input:

In [ ]:
 def parse_input(state: BaristaState) -> dict:
    raw = state.get("input_text")
    messages = state.get("messages", [])[:]
    if raw:
        messages.append(raw)

    order = state.get("order")
    confirmed = state.get("confirmed", False)
    menu_shown = state.get("menu_shown", False)

    lower = raw.lower() if raw else ""

    if "menu" in lower:
        menu_shown = True
    elif "confirm" in lower:
        confirmed = True
    elif "change" in lower:
        order = None
        confirmed = False
    elif order is None:
        drinks = ["espresso", "cappuccino", "latte", "green tea", "chai", "earl grey"]
        for drink in drinks:
            if drink in lower:
                order = drink.title()
                break

    return {
        "messages": messages,
        "order": order,
        "confirmed": confirmed,
        "menu_shown": menu_shown
    }

Display Menu (ToolNode)

In [ ]:
from langgraph.prebuilt import ToolNode

menu_node = ToolNode(tools=[menu_tool])

#6. Exit Condition#

Exit condition is checked:

Once the order is confirmed, the conversation is over, and the BaristaBot takes care of another customer.

On the contrary, if the order is not confirmed, conversation goes on.

In [ ]:
# Define the conditional logic function
def is_done(state: BaristaState) -> bool:
   # The conversation is done if the order is confirmed
    return state.get("confirmed", False)

#7. Define LangGraph Workflow#

At each step; workflow is managed thanks to LangGraph and its conversational Agents.

In [ ]:
from langgraph.graph import StateGraph, END

builder = StateGraph(BaristaState)

builder.add_node("parse_input", parse_input)
builder.set_entry_point("parse_input")

builder.add_edge("parse_input", END)  # Always end after one run
#builder.print_ascii() : imprimer le graphe (méthode non prise en charge avec cette version de Langgraph)
graph = builder.compile()

#8. Run BaristaBot (Main Loop)#

BaristaBot's conversational agent makes a loop:

- It asks the customer which beverage they will have.
- Customers make a choice, and,
- If they confirm their choice, conversation comes to an end.
- They can also modify their choice, before confirming, and coversation then comes to an end.
- The BaristaBot then moves on to another customer.

In [ ]:
# To run in a conversational loop, you can do something like this:
initial_state = BaristaState(
    messages=[],
    order=None,
    confirmed=False,
    menu_shown=False,
    input_text=""
)
state = initial_state
while not state["confirmed"]:
    user_input = input("You: ")
    # Pass the user_input in the state dictionary for the parse_input node
    # The graph will process this input and update the state

    result = graph.invoke({
        **state,
        "input_text": user_input
    })

    state.update(result)

    if state["menu_shown"]:
        print(get_menu())
        state["menu_shown"] = False

    if state["order"] and not state["confirmed"]:
        print(f"BaristaBot: You ordered a {state['order']}. Type 'confirm' to place it or 'change' to modify.")
    elif not state["menu_shown"]:
        print("BaristaBot: What would you like to order?")


You: Please, show me the menu
☕ Coffee Menu:
- Espresso
- Cappuccino
- Latte

🍵 Tea Menu:
- Green Tea
- Chai
- Earl Grey
BaristaBot: What would you like to order?
You: An espresso please
BaristaBot: You ordered a Espresso. Type 'confirm' to place it or 'change' to modify.
You: I want tot change
BaristaBot: What would you like to order?
You: A Chai
BaristaBot: You ordered a Chai. Type 'confirm' to place it or 'change' to modify.
You: confirm
BaristaBot: What would you like to order?


# ✅ **Completed**:

- A stateful LangGraph app
- Use of TypedDict for shared state
- Integration of Gemini API via LangChain
- Creation of a menu tool
- Use of ToolNode to handle dynamic responses
- Implementation of loops and conditional exits in LangGraph